## Introduction

Author's name: Rachel Mandelbaum (rmandelb on GitHub)

Date last tested: July 24, 2022

This notebook illustrates a comparison between the DP0.1 and DP0.2 object catalogs, with two goals:
1. Learning more about how to use the Table Access Protocol (TAP) server and ADQL (Astronomy Data Query Language) to query and retrieve data.
2. Comparing DP0.1 and DP0.2 object catalogs, to understand how differences in the image processing from v19 to v23 of the LSST Science Pipelines may affect the measured object properties.

Attribution: the elements of this notebook that involve learning how to query the object catalogs are heavily based on <a href="https://github.com/rubin-dp0/tutorial-notebooks/blob/main/02_Catalog_Queries_with_TAP.ipynb">one of the DP0.2 tutorials</a>.  Much of the explanatory text in the early parts of the notebook, before the comparison of DP0.1 and DP0.2 outputs, comes from there.


In [ ]:
# Import general python packages
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas
from pandas.testing import assert_frame_equal
from astropy import units as u
from astropy.coordinates import SkyCoord

# Import the Rubin TAP service utilities
from lsst.rsp import get_tap_service, retrieve_query

# To ignore some kinds of warnings
import warnings
from astropy.units import UnitsWarning

# Cosmetic: just make the plot font sizes a little larger
plt.rcParams.update({'font.size': 16})

In [ ]:
warnings.simplefilter("ignore", category=UnitsWarning)

## Explore the DP0.2 schema 

We start by getting an instance of the TAP service, and asserting that it exists, as in the tutorial linked in the header.

In [ ]:
service = get_tap_service()
assert service is not None
assert service.baseurl == "https://data.lsst.cloud/api/tap"

### Schema discovery

To find out what schemas, tables and columns exist, query the Rubin TAP schema.

This information is also available in the "Data Products Definitions" section of the <a href="dp0-2.lsst.io">DP0.2 documentation</a>.

Create the query to find out what schemas are in the Rubin TAP_SCHEMA, execute it, and see that a TAP Results object is returned.

This is pretty important because the schemas have changed considerably from DP0.1 to DP0.2.

In [ ]:
query = "SELECT * FROM tap_schema.schemas"
results = service.search(query)
print(type(results))

Convert the results to an astropy table and display

In [ ]:
results = service.search(query).to_table()
results

### 2.3. The DP0.2 catalogs

All the DP0 tables (catalogs) are in the "dp02_dc2_catalogs" schema (table collection).

Search for the DP0 schema name and store as a variable.

In [ ]:
schema_names = results['schema_name']
for name in schema_names:
    if name.find('dp01') > -1:
        dp01_schema_name = name
        break
print("DP0.1 schema is " + dp01_schema_name)
for name in schema_names:
    if name.find('dp02') > -1:
        dp02_schema_name = name
        break
print("DP0.2 schema is " + dp02_schema_name)


Let's explore tables in the DP0.1 and DP0.2 schema, ordering them by their database.
This is the order in which they will appear presented to the user in the RSP Portal.
We see the tables in the DP0.1 and DP0.2 schema, the same tables that are presented via the Portal GUI, together with a description of each. 

In [ ]:
query1 = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = '" \
        + dp01_schema_name + "' order by table_index ASC"
print(query1)
query2 = "SELECT * FROM tap_schema.tables " \
        "WHERE tap_schema.tables.schema_name = '" \
        + dp02_schema_name + "' order by table_index ASC"
print(query2)

results1 = service.search(query1)
results1 = results1.to_table()
results2 = service.search(query2)
results2 = results2.to_table()
results2


## Querying the DP0.1 and DP0.2 Object catalog

The Object catalogs (e.g., dp02_dc2_catalogs.Object) contain sources detected in the coadded images.

### 3.1. Getting the columns available for a given table

Request the column names, data types, descriptions, and units for all columns in the Object catalog, and display as a Pandas table (which will automatically truncate).

In [ ]:
results1 = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp01_dc2_catalogs.Object'")
results1.to_table().to_pandas()
results2 = service.search("SELECT column_name, datatype, description, unit from TAP_SCHEMA.columns "
                         "WHERE table_name = 'dp02_dc2_catalogs.Object'")
results2.to_table().to_pandas()

There is no need to read through all the columns, which are also available in the "Data Products Definitions" section of the <a href="dp0-2.lsst.io">DP0.2 documentation</a>.

The output of the next cell was used to identify the different column names for some of the information of interest in the DP0.1 and DP0.2 schema.

In [ ]:
print(results1['column_name'])
print(results2['column_name'])

Clean up.

In [ ]:
del results1,results2

### Cone search

A cone search on the Object table will be a common TAP query.
In this example, a circle centered on (RA, Dec) = (62.0, -37.0), with a radius of 0.2 degrees is used.  This area was chosen because it contains a reasonably large number of objects (around 40k) yet the queries still run in a 1 to several seconds.

Define the central coordinates and search radius using AstroPy `SkyCoord` and units.

In [ ]:
center_coords = SkyCoord(62, -37, frame='icrs', unit='deg')
search_radius = 0.2*u.deg

print(center_coords)
print(search_radius)

The TAP queries take the center coordinates and the search radius -- both in units of degrees -- as strings, so also define strings to use in the query statements below.

In [ ]:
use_center_coords = "62, -37"
use_radius = "0.2"

### Build and execute the query

Build queries to find objects down to a limiting magnitude of i<26, with basic flags set, in both the DP0.1 and DP0.2 object catalogs.  Note that not only column names differ; the manner of setting magnitude cuts, and the table names (object vs. Object) also differ.

In DP0.2 it is recommended to set `detect_isPrimary = True` (which means the source has no deblended children, to avoid returning both deblended *and* blended objects); in DP0.1 we use the clean flag.

Execute the queries; the two searches combined usually take about 5-10 seconds.

In [ ]:
%%time
query1 = "SELECT  " + \
        "objectId, ra, dec, clean, " + \
        "mag_i_cModel, mag_g_cModel, psf_fwhm_i, extendedness, blendedness " + \
        "FROM dp01_dc2_catalogs.object " + \
        "WHERE CONTAINS(POINT('ICRS', ra, dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND clean = 1 " + \
        "AND mag_i_cModel < 26.0"
results1 = service.search(query1)
print('DP0.1 query returned ',len(results1))

query2 = "SELECT  " + \
        "objectId, coord_ra, coord_dec, detect_isPrimary, detect_isIsolated, " + \
        "i_cModelFlux, g_cModelFlux, i_inputCount, i_fwhm, i_extendedness, i_blendedness, " + \
        "scisql_nanojanskyToAbMag(i_cModelFlux) as mag_i_cModel, " + \
        "scisql_nanojanskyToAbMag(g_cModelFlux) as mag_g_cModel " + \
        "FROM dp02_dc2_catalogs.Object " + \
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec), " + \
        "CIRCLE('ICRS', " + use_center_coords + ", " + use_radius + ")) = 1 " + \
        "AND detect_isPrimary = 1 " + \
        "AND scisql_nanojanskyToAbMag(i_cModelFlux) < 26.0"
results2 = service.search(query2)
print('DP0.2 query returned ',len(results2))


### Start to compare properties

We're going to start comparing object properties in DP0.1 vs. DP0.2.  To begin with, we just look at ensemble properties of the populations detected in this region, without matching on a per-object basis.

In [ ]:
# Define a utility that will let us look at a 2D histogram of object positions on the sky.
def show_radec(ra, dec, bins=30, title=None):
    plt.hist2d(ra, dec, bins)
    plt.gca().set_aspect('equal')
    plt.xlabel('RA [deg]')
    plt.ylabel('dec [deg]')
    if title is not None: plt.title(title)
    plt.colorbar()

We'll look at 2D histograms of object positions in DP0.1 and DP0.2.

In [ ]:
# Plot a 2d histogram of object positions, noting different column names for coordinate positions.
plt.figure(figsize=(15,7))
plt.subplot(121)
show_radec(results1['ra'], results1['dec'], title='DP0.1')
plt.subplot(122)
show_radec(results2['coord_ra'], results2['coord_dec'], title='DP0.2')

Observations:

1. The images look similar, in the sense that they have similar high/low points.
2. But they are not identical.  This is probably expected since we put a magnitude cut, and some objects may scatter across that limit due to differences in the image processing.

Next we will look at histograms of the g-i cmodel colors.

In [ ]:
plt.figure(figsize=(6,6))
plt.subplot(111)
plt.hist(results1['mag_g_cModel']-results1['mag_i_cModel'], 30, range=[-2,6], density=True, histtype='step', label='DP0.1')
plt.hist(results2['mag_g_cModel']-results2['mag_i_cModel'], 30, range=[-2,6], density=True, histtype='step', label='DP0.2')
plt.xlabel('g-i cmodel color')
plt.yscale('log')
plt.legend(loc='upper right')

These look very similar!  Now we'll do the same for the i-band magnitudes:

In [ ]:
plt.figure(figsize=(6,6))
plt.subplot(111)
plt.hist(results1['mag_i_cModel'], 30, range=[18,26], density=True, histtype='step', label='DP0.1')
plt.hist(results2['mag_i_cModel'], 30, range=[18,26], density=True, histtype='step', label='DP0.2')
plt.xlabel('i-band cmodel magnitude')
plt.yscale('log')
plt.legend(loc='upper left')

These also look very similar.  So at least the basic object properties -- positions, i-band magnitude, g-i color -- for the selected ensemble (40k objects) seem very similar.  We'll move on to comparison of per-object properties, which requires cross-matching.  I wasn't sure if there is a cross-matching routine in the RSP, so I just used astropy.

In [ ]:
import astropy.units as u
from astropy.coordinates import SkyCoord
c1 = SkyCoord(ra=results1['ra']*u.degree, dec=results1['dec']*u.degree)
c2 = SkyCoord(ra=results2['coord_ra']*u.degree, dec=results2['coord_dec']*u.degree)
max_sep = 1.0 * u.arcsec
idx, d2d, d3d = c1.match_to_catalog_sky(c2)
sep_constraint = d2d < max_sep
def get_matched_columns(cat1, cat2, mask1, mask2, colname1, colname2):
    return cat1[colname1][mask1], cat2[colname2][mask2]

# We'd like to check the per-object positions, and the i-band magnitudes.
# We'd further like to check these for isolated vs. non-isolated objects (as flagged by Scarlet in DP0.2).
# So we have to get a bunch of matched columns.
# Maybe there is a nice way to get fully reordered Tables but I couldn't get it to work in the time available to me.
matched_ra1, matched_ra2 = get_matched_columns(results1, results2, sep_constraint, idx[sep_constraint], 'ra', 'coord_ra')
matched_dec1, matched_dec2 = get_matched_columns(results1, results2, sep_constraint, idx[sep_constraint], 'dec', 'coord_dec')
matched_imag1, matched_imag2 = get_matched_columns(results1, results2, sep_constraint, idx[sep_constraint], 'mag_i_cModel', 'mag_i_cModel')
_, matched_isolated = get_matched_columns(results1, results2, sep_constraint, idx[sep_constraint], 'mag_g_cModel', 'detect_isIsolated')


We start by comparing the RA and dec values in DP0.2 vs. DP0.1, to look for astrometric offsets.

In [ ]:
import matplotlib as mpl
def show_deltaradec(ra1, dec1, ra2, dec2, bins=25, range=(-0.2,0.2), title=None):
    plt.hist2d(3600*(ra2-ra1)*np.cos(dec1*np.pi/180.),
               3600*(dec2-dec1), bins, norm=mpl.colors.LogNorm(),
               range=(range, range))
    plt.gca().set_aspect('equal')
    plt.axvline(0, c='red')
    plt.axhline(0, c='red')
    plt.colorbar(label='Number of objects')
    plt.xlabel(r'($\Delta$RA)cos(dec) [arcsec]')
    plt.ylabel(r'$\Delta$dec [arcsec]');
    if title is not None: plt.title(title)

plt.figure(figsize=(10,7))
show_deltaradec(matched_ra1, matched_dec1, matched_ra2, matched_dec2)

Given the log scale on the histogram, we can see that the astrometry agrees very well between DP0.1 and DP0.2.

Next, we'll explore the i-band cmodel magnitudes.  For this purpose, we'll plot mag(DP0.2)-mag(DP0.1) as a function of mag(DP0.1).  Besides the 2D histogram, we'll show lines for the median and 10th/90th percentiles, so we can quantify any systematic offsets and the spread in the distribution.

In [ ]:
import scipy.stats
x = matched_imag1
y = matched_imag2 - matched_imag1
x_range = [20, 26]

# Get running stats for Delta(mag)
def percentile10(y):
   return(np.percentile(y,10))
def percentile90(y):
   return(np.percentile(y,90))
running_median, edges, _ = scipy.stats.binned_statistic(x, y, statistic='median', bins=20, range=x_range)
running_10p, _, _ = scipy.stats.binned_statistic(x, y, statistic=percentile10, bins=20, range=x_range)
running_90p, _, _ = scipy.stats.binned_statistic(x, y, statistic=percentile90, bins=20, range=x_range)

# Now plot the 2D histogram along with the running stats.
plt.figure(figsize=(10,7))
plt.hist2d(x, y, 30, 
           range=[x_range, [-1,1]], norm=mpl.colors.LogNorm())
plt.axhline(0, c='red')
plt.plot(0.5*(edges[1:]+edges[:-1]), running_median, color='cyan', ls='--')
plt.plot(0.5*(edges[1:]+edges[:-1]), running_10p, color='cyan', ls='--')
plt.plot(0.5*(edges[1:]+edges[:-1]), running_90p, color='cyan', ls='--')
plt.colorbar(label='Number of objects')
plt.xlabel(r'i-band cmodel magnitude')
plt.ylabel(r'$\Delta$mag');

As shown, the median Delta(mag) is very close to 0.  However, the 10th/90th percentiles indicate significant scatter, even though these object catalogs are based on the same image pixels.  Presumably this reflects some evolution in the LSST Science Pipelines detection, selection, deblending, or cmodel magnitude measurement algorithm.

Following a suggestion from Melissa Graham, I decided to check whether it could be due to the new deblender (Scarlet), by comparing the 10th/90th percentiles and the median for those objects that Scarlet flagged as isolated vs. those that were not. Presumably the isolated ones may not have been deblended in either DP0.1 or DP0.2, so they should agree if the deblender is responsible for the above differences.

In [ ]:
x_iso = matched_imag1[matched_isolated]
y_iso = matched_imag2[matched_isolated] - matched_imag1[matched_isolated]
x_not_iso = matched_imag1[~matched_isolated]
y_not_iso = matched_imag2[~matched_isolated] - matched_imag1[~matched_isolated]
print('Isolated, non-isolated, and total matched objects:',len(x_iso),len(x_not_iso), len(matched_imag1))

running_median_iso, edges_iso, _ = scipy.stats.binned_statistic(x_iso, y_iso, statistic='median', bins=10, range=x_range)
running_10p_iso, _, _ = scipy.stats.binned_statistic(x_iso, y_iso, statistic=percentile10, bins=10, range=x_range)
running_90p_iso, _, _ = scipy.stats.binned_statistic(x_iso, y_iso, statistic=percentile90, bins=10, range=x_range)
running_median_not_iso, edges_not_iso, _ = scipy.stats.binned_statistic(x_not_iso, y_not_iso, statistic='median', bins=20, range=x_range)
running_10p_not_iso, _, _ = scipy.stats.binned_statistic(x_not_iso, y_not_iso, statistic=percentile10, bins=20, range=x_range)
running_90p_not_iso, _, _ = scipy.stats.binned_statistic(x_not_iso, y_not_iso, statistic=percentile90, bins=20, range=x_range)

plt.figure(figsize=(10,7))
plt.plot(0.5*(edges_iso[1:]+edges_iso[:-1]), running_median_iso, color='magenta', label='Isolated')
plt.plot(0.5*(edges_iso[1:]+edges_iso[:-1]), running_10p_iso, color='magenta', ls='--')
plt.plot(0.5*(edges_iso[1:]+edges_iso[:-1]), running_90p_iso, color='magenta', ls='--')
plt.plot(0.5*(edges_not_iso[1:]+edges_not_iso[:-1]), running_median_not_iso, color='blue', label='Not isolated')
plt.plot(0.5*(edges_not_iso[1:]+edges_not_iso[:-1]), running_10p_not_iso, color='blue', ls='--')
plt.plot(0.5*(edges_not_iso[1:]+edges_not_iso[:-1]), running_90p_not_iso, color='blue', ls='--')
#plt.axhline(0, color='black')
plt.xlabel(r'i-band cmodel magnitude')
plt.ylabel(r'$\Delta$mag')
plt.legend()
plt.ylim((-0.4,0.4))

As indicated, 25% of the objects are flagged as isolated.  Those have a very narrow scatter (10th/90th percentiles indicated as dashed lines).  The non-isolated objects have a much broader scatter, suggesting that the differences in cmodel magnitudes from DP0.1 to DP0.2 may be primarily because of the different deblender.

This plot led me to wonder if the astrometric scatter has the same origin, so I remade the astrometry difference plot for isolated vs. non-isolated objects:

In [ ]:
plt.figure(figsize=(15,7))
plt.subplot(121)
show_deltaradec(matched_ra1[matched_isolated], matched_dec1[matched_isolated],
                matched_ra2[matched_isolated], matched_dec2[matched_isolated],
                title='Isolated')
plt.subplot(122)
show_deltaradec(matched_ra1[~matched_isolated], matched_dec1[~matched_isolated],
                matched_ra2[~matched_isolated], matched_dec2[~matched_isolated],
                title='Not isolated')

While not a quantitative comparison, visually we can see that the central pixels have similar values for the two cases, but the right plot (non-isolated) has more spread.  So qualitatively, at least, it appears that the difference in deblending may explain a good fraction of the astrometric offsets between detected centroids in the object catalogs.